In [1]:
#!pip install flax diffusers transformers ftfy stable_diffusion_videos yt-dlp -q
import jax; jax.devices()

[CpuDevice(id=0)]

In [2]:
#!pip install --upgrade jax jaxlib 

import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu('tpu_driver_20221011')

#!pip install flax diffusers transformers ftfy
jax.devices()

RuntimeError: jax.tools.colab_tpu.setup_tpu() was required for older JAX versions running on older generations of TPUs, and should no longer be used.

In [ ]:
#!apt -qq update
#!apt -qq install libgl1-mesa-glx ffmpeg -y

In [2]:
video_duration = 45 # time in seconds of the video
fps = 30
repeat_prompts_every = 2 # time in seconds of how long to wait between prompts before a different image using the same prompt is generated 
time_per_transition = 0.5 # time in seconds to transition between different prompts

text_transcript = [
      # PUT TEXT TRANSCRIPT HERE
(0, 'Goat'),
(1.68, 'animal'),
(2.16, 'water'),
(3.08, 'goat man'),
(5.2, 'jumping from the city skyline'),
(9.04, 'Jesus Christ'),
(10.68, 'keine'),
(11.38, 'beiden Sprachen'),
(15.2, 'irgendwelchen Worten'),
(19.26, 'Obama'),
(19.86, 'Bush'),
(19.98, 'nein'),

]

In [1]:
import numpy as np
import jax, random
import jax.numpy as jnp
from flax.jax_utils import replicate

from stable_diffusion_videos import FlaxStableDiffusionWalkPipeline, Interface

pipeline, params = FlaxStableDiffusionWalkPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", 
    revision="bf16", 
    dtype=jnp.bfloat16,
    safety_checker=None
)
p_params = replicate(params)

d:\Anaconda\envs\stable_diff\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: AsyncConnectionPool.__init__() got an unexpected keyword argument 'socket_options'

In [ ]:
prompts = []
num_interpolation_steps = []

for idx, (start, text) in enumerate(text_transcript):
    if idx == len(text_transcript) - 1:
        end = video_duration
    else:
        end = text_transcript[idx+1][0]

    duration = end - start
    prompt_repeats = int((duration - time_per_transition) // repeat_prompts_every)
    remainder_prompt_time = (duration - time_per_transition) % repeat_prompts_every

    prompts += [text] * prompt_repeats
    num_interpolation_steps += [fps * repeat_prompts_every] * prompt_repeats

    if int(remainder_prompt_time * fps) != 0:
        prompts.append(text)
        num_interpolation_steps.append(int(remainder_prompt_time * fps))

    num_interpolation_steps.append(fps//2)
    prompts.append(text)

num_interpolation_steps = num_interpolation_steps[:-1]
num_interpolation_steps[-1] += int(time_per_transition * fps)

seeds = [random.randint(0, 100000) for _ in range(len(prompts))]
print(prompts)
print(num_interpolation_steps)
print(seeds)

In [ ]:
def visualize_video_local(video_path, save_dir="videos"):
    import os
    os.makedirs(save_dir, exist_ok=True)
    local_save_path = os.path.join(save_dir, os.path.basename(video_path))
    with open(video_path, 'rb') as f:
        data = f.read()
    with open(local_save_path, 'wb') as f:
        f.write(data)
    print(f"Video saved locally at {local_save_path}")
    return local_save_path

In [ ]:
import time

tic = time.time()

video_path = pipeline.walk(
    p_params,
    prompts,
    seeds,
    fps=fps,                      # use 5 for testing, 25 or 30 for better quality
    num_interpolation_steps=num_interpolation_steps,  # use 3-5 for testing, 30 or more for better results
    output_dir='./dreams',                 # Where images will be saved
    jit=True,                    # To use all TPU cores,
    guidance_scale=7.5,         # Higher adheres to prompt more, lower lets model take the wheel
    num_inference_steps=20,
)
    
toc = time.time()
print(video_path)
print('Time in seconds to generate video:', toc - tic)
visualize_video_local(video_path)

In [3]:
import random

import torch
from stable_diffusion_videos import StableDiffusionWalkPipeline
from diffusers.utils.import_utils import is_xformers_available
from diffusers.models import AutoencoderKL
from diffusers.schedulers import LMSDiscreteScheduler


pipe = StableDiffusionWalkPipeline.from_pretrained(
    'runwayml/stable-diffusion-v1-5',
    torch_dtype=torch.float16,
    safety_checker=None,
    vae=AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda"),
    scheduler=LMSDiscreteScheduler(
        beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear"
    )
).to("cuda")


if is_xformers_available():
    pipe.enable_xformers_memory_efficient_attention()

# I give you permission to scrape this song :)
# youtube-dl -f bestaudio --extract-audio --audio-format mp3 --audio-quality 0 -o "music/thoughts.%(ext)s" https://soundcloud.com/nateraw/thoughts
audio_filepath = 'music/thoughts.mp3'

# Seconds in the song. Here we slice the audio from 0:07-0:16
# Should be same length as prompts/seeds.
audio_offsets = [7, 10, 13]

# Output video frames per second.
# Use lower values for testing (5 or 10), higher values for better quality (30 or 60)
fps = 25

# Convert seconds to frames
# This array should be `len(prompts) - 1` as its steps between prompts.
num_interpolation_steps = [(b-a) * fps for a, b in zip(audio_offsets, audio_offsets[1:])]

prompts = ["a cat with a funny hat", "snoop dogg at the dmv", "steak flavored ice cream"]
seeds = [random.randint(0, 9e9) for _ in range(len(prompts))]

pipe.walk(
    prompts=prompts,
    seeds=seeds,
    num_interpolation_steps=num_interpolation_steps,
    fps=fps,
    audio_filepath=audio_filepath,
    audio_start_sec=audio_offsets[0],
    batch_size=12,  # Increase/decrease based on available GPU memory. This fits on 24GB A10
    num_inference_steps=50,
    guidance_scale=15,
    margin=1.0,
    smooth=0.2,
)

ImportError: cannot import name 'StableDiffusionWalkPipeline' from 'stable_diffusion_videos' (d:\Anaconda\envs\stable_diff\lib\site-packages\stable_diffusion_videos\__init__.py)